# Baseline Popular Recommender
As with any ML project, we start with a non-ML approach as our baseline. It's very common in RecSys project to have popular recommender as not only a benchmark but also an actual component (retrieval stage).

# Set up

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

import mlflow
import pandas as pd
from dotenv import load_dotenv
from loguru import logger
from pydantic import BaseModel

load_dotenv()

sys.path.insert(0, "..")

from src.eval import (
    create_label_df,
    log_classification_metrics,
    log_ranking_metrics,
    merge_recs_with_target,
)
from src.viz import blueq_colors

# Controller

In [3]:
class Args(BaseModel):
    testing: bool = False
    log_to_mlflow: bool = True
    experiment_name: str = "FSDS RecSys - L4 - Reco Algo"
    run_name: str = "001-baseline-popular"
    notebook_persist_dp: str = None
    random_seed: int = 41

    user_col: str = "user_id"
    item_col: str = "parent_asin"
    rating_col: str = "rating"
    timestamp_col: str = "timestamp"

    top_K: int = 100
    top_k: int = 10

    batch_size: int = 128

    def init(self):
        self.notebook_persist_dp = os.path.abspath(f"data/{self.run_name}")

        if not os.environ.get("MLFLOW_TRACKING_URI"):
            logger.warning(
                f"Environment variable MLFLOW_TRACKING_URI is not set. Setting self.log_to_mlflow to false."
            )
            self.log_to_mlflow = False

        if self.log_to_mlflow:
            logger.info(
                f"Setting up MLflow experiment {self.experiment_name} - run {self.run_name}..."
            )

            mlflow.set_experiment(self.experiment_name)
            mlflow.start_run(run_name=self.run_name)

        return self


args = Args().init()

print(args.model_dump_json(indent=2))

2024-09-24 06:24:15.590 | INFO     | __main__:init:29 - Setting up MLflow experiment FSDS RecSys - L4 - Reco Algo - run 001-baseline-popular...
2024/09/24 06:24:15 INFO mlflow.tracking.fluent: Experiment with name 'FSDS RecSys - L4 - Reco Algo' does not exist. Creating a new experiment.


{
  "testing": false,
  "log_to_mlflow": true,
  "experiment_name": "FSDS RecSys - L4 - Reco Algo",
  "run_name": "001-baseline-popular",
  "notebook_persist_dp": "/home/jupyter/frostmourne/reco-algo/notebooks/data/001-baseline-popular",
  "random_seed": 41,
  "user_col": "user_id",
  "item_col": "parent_asin",
  "rating_col": "rating",
  "timestamp_col": "timestamp",
  "top_K": 100,
  "top_k": 10,
  "batch_size": 128
}


# Prep data

In [4]:
train_df = pd.read_parquet("../data/train_features_neg_df.parquet")
val_df = pd.read_parquet("../data/val_features_neg_df.parquet")

# Implement

In [5]:
popular_items_df = (
    train_df.groupby(args.item_col, as_index=False)
    .size()
    .assign(
        score=lambda df: df["size"] / df["size"].max(),
        rec_ranking=lambda df: df["score"]
        .rank(method="first", ascending=False)
        .astype(int),
    )
    .sort_values(["rec_ranking"], ascending=[True])
)

top_popular_items_df = popular_items_df.head(args.top_K)

top_popular_items_df

,parent_asin,size,score,rec_ranking
2143,B0086VPUHI,1530,1.000000,1
3554,B01N3ASPNV,1484,0.969935,2
4337,B07YBXFDYN,1441,0.941830,3
2368,B00BGA9WK2,1261,0.824183,4
1775,B004RMK5QG,1120,0.732026,5
...,...,...,...,...
2149,B00884S5N4,346,0.226144,96
4330,B07YBX7Y3P,343,0.224183,97
947,B001EYUQVE,342,0.223529,98
1719,B004I1JTEK,341,0.222876,99


In [6]:
recommendations_df = (
    val_df[[args.user_col]]
    .drop_duplicates()
    .assign(tmp=1)
    .pipe(
        lambda df: pd.merge(
            df, top_popular_items_df.assign(tmp=1), on="tmp", how="left"
        )
    )[[args.user_col, args.item_col, "score", "rec_ranking"]]
)

recommendations_df

,user_id,parent_asin,score,rec_ranking
0,AGXTHABHPC3XO4VAMCFM2TQR3GFQ,B0086VPUHI,1.000000,1
1,AGXTHABHPC3XO4VAMCFM2TQR3GFQ,B01N3ASPNV,0.969935,2
2,AGXTHABHPC3XO4VAMCFM2TQR3GFQ,B07YBXFDYN,0.941830,3
3,AGXTHABHPC3XO4VAMCFM2TQR3GFQ,B00BGA9WK2,0.824183,4
4,AGXTHABHPC3XO4VAMCFM2TQR3GFQ,B004RMK5QG,0.732026,5
...,...,...,...,...
70795,AGKKMX2QPFELPACFFPUBWRO55KNQ,B00884S5N4,0.226144,96
70796,AGKKMX2QPFELPACFFPUBWRO55KNQ,B07YBX7Y3P,0.224183,97
70797,AGKKMX2QPFELPACFFPUBWRO55KNQ,B001EYUQVE,0.223529,98
70798,AGKKMX2QPFELPACFFPUBWRO55KNQ,B004I1JTEK,0.222876,99


# Evaluate

## Ranking metrics

In [7]:
label_df = create_label_df(
    val_df,
    user_col=args.user_col,
    item_col=args.item_col,
    rating_col=args.rating_col,
    timestamp_col=args.timestamp_col,
)
label_df

,user_id,parent_asin,rating,rating_rank
1727,AEOY2365QPPEVDTOXL6N7ZA4NSAA,B00PDRZG9U,5.0,1.0
451,AFGHX4VLP6P5XORLDJX3LZKUAAZA,B00Z9TJBUW,5.0,1.0
204,AFCH2PDOFM2S3622QFV6PHCHGMCA,B00KSQHX1K,5.0,1.0
1344,AEURBISVS35ALE7YQLR5L4K7AHCA,B07QQ8N7LL,1.0,1.0
334,AEMA3SW3WPNLEH3IACW23K2ZSUFA,B09JDLC31H,4.0,1.0
...,...,...,...,...
1332,AFB6FYPPCN33UMUU5536IHXNOHCQ,B002I0K3CK,0.0,18.0
910,AESD4RLWUKM6JTD6SNNWYLHLLQQA,B07BLRF329,0.0,18.0
695,AG4RCXKPTC6QRORJLUSBY4SO2IAA,B001ELJDWA,0.0,18.0
1177,AFB6FYPPCN33UMUU5536IHXNOHCQ,B003S6N7OO,0.0,19.0


In [8]:
eval_df = merge_recs_with_target(
    recommendations_df,
    label_df,
    k=args.top_K,
    user_col=args.user_col,
    item_col=args.item_col,
    rating_col=args.rating_col,
)
eval_df

,user_id,parent_asin,score,rec_ranking,rating,rating_rank
25,AE2AZ2MNROPF33U6SS53VI22OXJA,B0086VPUHI,1.000000,1,0,NaN
79,AE2AZ2MNROPF33U6SS53VI22OXJA,B01N3ASPNV,0.969935,2,0,NaN
89,AE2AZ2MNROPF33U6SS53VI22OXJA,B07YBXFDYN,0.941830,3,0,NaN
31,AE2AZ2MNROPF33U6SS53VI22OXJA,B00BGA9WK2,0.824183,4,0,NaN
18,AE2AZ2MNROPF33U6SS53VI22OXJA,B004RMK5QG,0.732026,5,0,NaN
...,...,...,...,...,...,...
72374,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B001EYUQVE,0.223529,98,0,NaN
72384,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B004I1JTEK,0.222876,99,0,NaN
72414,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B00DBLBMBQ,0.222876,100,0,NaN
72367,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B00004U1R2,NaN,101,0,2.0


In [9]:
ranking_report = log_ranking_metrics(args, eval_df)

## Classification metrics

In [10]:
eval_classification_df = pd.merge(
    val_df,
    popular_items_df[[args.item_col, "score"]],
    on=[args.item_col],
    how="left",
    validate="m:1",
).assign(label=lambda df: df[args.rating_col].gt(0).astype(int))
eval_classification_df

,user_id,parent_asin,rating,timestamp,user_indice,item_indice,main_category,title,description,categories,price,item_sequence,score,label
0,AGXTHABHPC3XO4VAMCFM2TQR3GFQ,B00TEDK8FQ,0.0,1643101921864,12853,2636,Video Games,Ortz PS4 Vertical Stand with Cooling Fan [Keep...,[],"[Video Games, PlayStation 4, Accessories, Case...",None,"[111, 3920, 3879, 3261, 3402, 1230, 2239, 3974...",0.076471,0
1,AESD4RLWUKM6JTD6SNNWYLHLLQQA,B07BMRGKX2,0.0,1653590691326,141,253,Video Games,Agony - PlayStation 4,"[Agony is a first-person, survival horror game...","[Video Games, PlayStation 4, Games]",28.0,"[99, 4672, 4434, 1551, 1561, 2497, 3615, 3196,...",0.016993,0
2,AEXFEQ7QOP6EHDEZ3K6NN27MQ7KA,B0774N9JKW,0.0,1651718479413,13969,1217,Video Games,Sword Art Online: Hollow Realization - PlaySta...,"[""Link start"" into SWORD ART ONLINE -Hollow Re...","[Video Games, PlayStation 4, Games]",18.11,"[1016, 3999, 2944, 742, 3161, 3580, 2267, 3623...",0.016993,0
3,AGDAPPCYV472FOUKDGAHZRW766GA,B07B416X7V,0.0,1649310595659,18594,3519,Video Games,Burnout Paradise Remastered - Xbox One [Digita...,[Make action your middle name as you rule the ...,"[Video Games, Xbox One, Games]",None,"[3097, 2497, 3203, 3937, 3803, 2323, 2310, 751...",0.013725,0
4,AG2KBJG5DMEIISPJVF3OVMRB4ALA,B001D8Q5MA,0.0,1636861380056,5042,3122,Video Games,Grand Theft Auto IV [Online Game Code],"[From the Manufacturer, What does the American...","[Video Games, PC]",None,"[-1, -1, -1, -1, -1, 3602, 22, 1193, 2486, 1293]",0.292810,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1893,AGVBR47VU2BE4EVWFAXOO26SOWSA,B0C39GFK7P,1.0,1647361584062,388,4128,Computers,"Logitech G640 Large Cloth Gaming Mouse Pad, Op...",[The cloth surface of G640 provides ideal surf...,"[Video Games, PC, Accessories, Gaming Mice]",29.99,"[1938, 2662, 262, 3903, 3610, 1896, 1372, 4160...",0.038562,1
1894,AFUWPAK6VCGEL2OVIL2YGZNFQJZQ,B08N6NCR3Q,4.0,1642699950266,4205,3269,Video Games,Thrustmaster T 16000M SPACE SIM DUO STICK (PC),[The THRUSTMASTER T.16000M FCS Space Sim Duo c...,"[Video Games, PC, Accessories, Controllers, Fl...",119.51,"[-1, -1, -1, -1, 1058, 3558, 377, 1187, 2169, ...",0.047712,1
1895,AFH63KLSVQQYRNFS7NLQGD3GSP3A,B094YHB1QK,5.0,1652564728981,20004,4190,Video Games,PlayStation DualSense Wireless Controller – Ga...,[Plot a course for astronomical adventures on ...,"[Video Games, PlayStation 5, Accessories, Cont...",74.99,"[-1, 832, 3126, 1490, 4335, 2035, 1270, 605, 3...",0.079739,1
1896,AFPPTJOEUPVXA5C63SNRGID3EQNA,B0BVVTQ5JP,4.0,1635968491390,4984,2257,Computers,Logitech G502 HERO High Performance Wired Gami...,[Logitech updated its iconic G502 gaming mouse...,"[Video Games, PC, Accessories, Gaming Mice]",45.87,"[-1, -1, -1, -1, -1, 4269, 4366, 396, 3060, 464]",0.077124,1


In [11]:
eval_classification_df["score"].describe().T

count    1898.000000
mean        0.110552
std         0.154800
min         0.009150
25%         0.026144
50%         0.054902
75%         0.119608
max         1.000000
Name: score, dtype: float64

In [12]:
classification_report = log_classification_metrics(
    args, eval_classification_df, target_col="label", prediction_col="score"
)

## Loss
We can estimate what kinds of MSE loss should be the upper bound model training based on measuring the loss when asking the model to naively predict the mean rating for every item.

In [13]:
naive_prediction = train_df[args.rating_col].mean()

naive_mse = (
    (
        val_df[args.rating_col]
        - val_df.assign(naive_prediction=naive_prediction)["naive_prediction"]
    )
    .apply(lambda s: s * s)
    .mean()
)
logger.info(
    f"Val MSE = {naive_mse:,.2f} given naive_prediction={naive_prediction:,.1f}"
)

2024-09-24 06:24:24.270 | INFO     | __main__:<module>:11 - Val MSE = 5.12 given naive_prediction=2.2


# Clean up

In [14]:
all_params = [args]

if args.log_to_mlflow:
    for params in all_params:
        params_dict = params.dict()
        params_ = {f"{params.__repr_name__()}.{k}": v for k, v in params_dict.items()}
        mlflow.log_params(params_)

    mlflow.end_run()

2024/09/24 06:24:24 INFO mlflow.tracking._tracking_service.client: 🏃 View run 001-baseline-popular at: http://localhost:5003/#/experiments/2/runs/a350e1f5fab0427fbb192bb01f50e64a.
2024/09/24 06:24:24 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5003/#/experiments/2.
